In [1]:
import os
import sys
from pandas_datareader import data as pdr
import pandas as pd
import fix_yahoo_finance as yf
import importlib
sys.path.append("../MachineLearningStocks/")
from download_historical_prices import *
from parsing_keystats import *
from backtesting import *
from stock_prediction import *
sys.path.append("../awsQLDB/")
os.environ["AWS_PROFILE"] = "Insight"
from create_ledger import *
from connect_to_ledger import *
from create_table import *
from create_index import * 
from insert_document import *
from hashlib import sha256
import pickle
sys.path.append("./")

/Users/kenmiyachi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
sp500 = build_sp500_dataset()

[*********************100%***********************]  1 of 1 downloaded


In [ ]:
sp500_df, stock_df = preprocess_price_data()
parsed_df = parse_keystats(sp500_df, stock_df)

In [ ]:
backtest()
# check_yahoo()
# current_df = forward()
# current_df.to_csv("forward_sample.csv", index=False)

In [ ]:
predict_stocks()

In [4]:
def collectionData(inputData, outHash):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    collection_data = [
        {
            'GitHash': commit_hash,
            'ScrapeTime': datetime.now(),
            'InputData': inputData,
            'outputHash': outHash
        }
    ]
    return collection_data

def trainingData(inputHash, model):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    training_data = [
        {
            'GitHash': commit_hash,
            'TrainTime': datetime.now(),
            'InputHash': inputHash,
            'Model': model
        }
    ]
    return training_data

def predictionData(inputHash, model, outHash):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    prediction_data = [
        {
            'GitHash': commit_hash,
            'InputHash': inputHash,
            'Model': model, 
            'PredictionTime': datetime.now(),
            'OutputHash': outHash
        }
    ]
    return prediction_data

In [5]:
import random
def mess_up(x):
    return x*random.random()
    

def data_collection(source="Yahoo Finance"):
    sp500 = build_sp500_dataset()
    sp500['Open'] = sp500['Open'].apply(lambda x: mess_up(x))
    sp500['Close'] = sp500['Close'].apply(lambda x: mess_up(x))
    sp500.to_csv("sp500_index.csv")
    s = pickle.dumps(sp500)
    outputHash = sha256(s).hexdigest()
    print(outputHash)
    # sp500_df, stock_df = preprocess_price_data()
    # parse_keystats(sp500_df, stock_df)
    data = collectionData(source, outputHash)
    dynamic_insert("StockMarketPrediction", "Collection", data)


def training(modelName="RandomForest-2020"): 
    model = backtest()
    s = pickle.dumps(model)
    modelHash = sha256(s).hexdigest()
    data = trainingData(modelHash, modelName)
    dynamic_insert("StockMarketPrediction", "Training", data)

def predict(modelName="RandomForest-2020"):
    stats = pd.read_csv('./new_keystats.csv')
    s = pickle.dumps(stats)
    inputHash = sha256(s).hexdigest()
    prediction = predict_stocks()
    outputHash = sha256(str.encode(prediction)).hexdigest()
    data = predictionData(inputHash, modelName, outputHash)
    dynamic_insert("StockMarketPrediction", "Prediction", data)

In [6]:
data_collection()
training("New Model RF")
predict("New Model RF")

[*********************100%***********************]  1 of 1 downloaded
ee8c92f9c2f4a30b5de6349c4556752ae7dc0dc44911a094bbf5e47892b50868
INFO:insert_document:Inserting some documents in the Collection table...
INFO:insert_document:Documents inserted successfully!
Classifier performance
Accuracy score:  0.79
Precision score:  0.78

 Stock prediction performance report 
Total Trades: 197
Average return for stock predictions:  37.5 %
Average market return in the same period:  8.6% 
Compared to the index, our strategy earns  28.9 percentage points more
INFO:insert_document:Inserting some documents in the Training table...
INFO:insert_document:Documents inserted successfully!
13 stocks predicted to outperform the S&P500 by more than 10%:
TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD
INFO:insert_document:Inserting some documents in the Prediction table...
INFO:insert_document:Documents inserted successfully!


In [7]:
ks = pd.read_csv('./new_keystats.csv')
ks['Price'] = ks['Price'].apply(lambda x: mess_up(x))
ks['SP500'] = ks['SP500'].apply(lambda x: mess_up(x))